<h1 align="center">Block Validation</h1>
<h2 align="center">Bruno Gonçalves</h2>
<h4 align="center">bgoncalves@gmail.com</h4>
<h4 align="center">@bgoncalves</h4>

In [1]:
import hashlib
import codecs
import binascii
from pprint import pprint

import json

Let's get a block we can trust. [Blockchain.com](https://blockchain.com) has a full featured [API](https://www.blockchain.com/api) that makes this particularly easy. For ease of reference, the public facing pages are hosted at blockchain.**COM** while the API endpoints live at blockchain.**INFO**

To avoid connectivity issues, and for reproducibility, I downloaded the block at height 150,000 previously

In [2]:
try:
    import requests
    url = "https://blockchain.info/rawblock/0000000000000a3290f20e75860d505ce0e948a1d1d846bec7e39015d242884b"

    req = requests.get(url)
    block = req.json()
except:
    block = json.loads(open("data/block_150000.json", "rt").readlines()[0])

As we can see, the block contains all the information we discussed in the slides, plus some values that were precomputed by our friends at [blockchain.com](https://blockchain.com)

In [3]:
pprint(block)

{'bits': 436956491,
 'block_index': 164971,
 'fee': 0,
 'hash': '0000000000000a3290f20e75860d505ce0e948a1d1d846bec7e39015d242884b',
 'height': 150000,
 'main_chain': True,
 'mrkl_root': 'be0b136f2f3db38d4f55f1963f0acac506d637b3c27a4c42f3504836a4ec52b1',
 'n_tx': 10,
 'next_block': ['000000000000006b1c90f01554b0aa0b2b3adbe339b175c23b8b8ce8b503a89f'],
 'nonce': 1796110725,
 'prev_block': '00000000000008df4269884f1d3bfc2aed3ea747292abb89be3dc3faa8c5d26f',
 'size': 3766,
 'time': 1319118291,
 'tx': [{'block_height': 150000,
         'block_index': 164971,
         'double_spend': False,
         'fee': 0,
         'hash': 'bcdc61cbecf6137eec5c8ad4047fcdc36710e77e404b17378a33ae605920afe1',
         'inputs': [{'script': '044b6d0b1a020b02',
                     'sequence': 4294967295,
                     'witness': ''}],
         'lock_time': 0,
         'out': [{'addr': '13PHR5QM2cJLkFoA6E3rPEwTyYxxSCJ3B4',
                  'n': 0,
                  'script': '4104e8e37f1556b53b557405fc79

We can also take advantage of Firefoxs particularly nice support for JSON files by opening the cached file directly [data/block_150000.json](data/block_150000.json)

# Bit Twiddling

In order to minimize the amount of data that needs to be transmitted and stored, BitCoin uses a binary format. As such, there are some conventions we need to follow and some helper functions that need to be defined

In particular, BitCoin stores values in 'little endian' format so we must often invert the byteorder to be able to compare the raw values with the published ones.

In [4]:
def invert_byteorder(original, decode_hex = codecs.getdecoder("hex_codec")):
    return decode_hex(original)[0][::-1].hex()

As an added security measure, Bitcoin uses a double SHA256 hashing function that we define below

In [5]:
def doubleSha256(input):
    decode_hex = codecs.getdecoder("hex_codec")
    input_bin = decode_hex(input)[0]
    return hashlib.sha256(hashlib.sha256(input_bin).digest()).hexdigest()

# Calculate the Merkle Tree root

Let's start by calculating the merkle tree root of all the transactions contained in this block. For ease of refernce, the transaction hashes are:

In [6]:
transactions = []

for tx in block["tx"]:
    transactions.append(invert_byteorder(tx["hash"]))

In [7]:
pprint(transactions)

['e1af205960ae338a37174b407ee71067c3cd7f04d48a5cec7e13f6eccb61dcbc',
 'a314970cd7c647d1cc0a477e1a2122b98205b6924b73001b8dab20ee81c2f4f7',
 'b08eb9dce0452a1b1970c4d29e88bdee07669a2a5d1b08586d7ffa17b2e3f6b5',
 '958b9e94aea9a485ba494c50fb3192558057f7caed9705d4b11369f071f10642',
 '3d278ca01cc527d4c7d577b668e0b976fbbcb94e6a9ba89664c721f36fada6a1',
 '95affa405501498ef6f4e4b6cddc1f1b24b9d2e534584b31a9142c453920c889',
 'd97a21cf46fd5afb0bf9ea4237bc4bf5c84e8b47d38d1eee2bbeb5c0f8a1c625',
 'ae1e670bdbf8ab984f412e6102c369aeca2ced933a1de74712ccda5edaf4ee57',
 '90e03319ddc9d48da38ab39b2f37c0a5af5afc736f6ff2a9d8b8653e0feb308d',
 '84251842a4c0f0e188e1c2bf643ec37a1402dd86a25a9ab5004633467d16e313']


As we saw, we have to calcuate the double hash of concated pairs of transactions until we reach the root of the tree. We also print out some intermediat results for clarity

In [8]:
def get_root(transactions):
    missing = len(transactions)

    for i in range(missing):
        print(i, transactions[i])

    next_transactions = []

    while missing > 1:
        print(missing)
        next_transactions = []

        # If it's odd, repeat the last transaction
        if missing % 2 == 1:
            transactions.append(transactions[-1])
            missing += 1

        for i in range(0, missing, 2):
            input = transactions[i] + transactions[i+1]
            output = doubleSha256(input)
            next_transactions.append(output)
            print(i, i+1, next_transactions[-1])


        missing = len(next_transactions)
        transactions = next_transactions[::]

    return invert_byteorder(next_transactions[0])

In [9]:
root = get_root(transactions)

0 e1af205960ae338a37174b407ee71067c3cd7f04d48a5cec7e13f6eccb61dcbc
1 a314970cd7c647d1cc0a477e1a2122b98205b6924b73001b8dab20ee81c2f4f7
2 b08eb9dce0452a1b1970c4d29e88bdee07669a2a5d1b08586d7ffa17b2e3f6b5
3 958b9e94aea9a485ba494c50fb3192558057f7caed9705d4b11369f071f10642
4 3d278ca01cc527d4c7d577b668e0b976fbbcb94e6a9ba89664c721f36fada6a1
5 95affa405501498ef6f4e4b6cddc1f1b24b9d2e534584b31a9142c453920c889
6 d97a21cf46fd5afb0bf9ea4237bc4bf5c84e8b47d38d1eee2bbeb5c0f8a1c625
7 ae1e670bdbf8ab984f412e6102c369aeca2ced933a1de74712ccda5edaf4ee57
8 90e03319ddc9d48da38ab39b2f37c0a5af5afc736f6ff2a9d8b8653e0feb308d
9 84251842a4c0f0e188e1c2bf643ec37a1402dd86a25a9ab5004633467d16e313
10
0 1 a4a2774e14677eaf13a5e8d5f793618ee3b9763ebbd99ac20894b2cea5aa17b7
2 3 40213c81f059806fb8c1b91d0a7397a57156cfc3a17b71d095c244aafc1eb115
4 5 efc2b3db87ff4f00c79dfa8f732a23c0e18587a73a839b7710234583cdd03db9
6 7 066ad6d9939ec0c90b7f3b775122785bd8aca2a2c5857205af7e0615e9af9796
8 9 11c42391000d1c9a2211a74d4f62be958927e905e04f837

Confirm we got the correct result

In [10]:
root == block['mrkl_root']

True

# Verify that a given transaction is part of the Merkle Tree

One of the advantages of a merkle tree is that it allows use to easily verify if a given transaction is part of a tree or not, without having to know the hashes of all the transactions. Indeed, we must only supply the transaction we want to verify and the path up the tree, resulting in $log_2(N)$ calculations, a significant improvement for large trees.

In [11]:
proof = ['AE1E670BDBF8AB984F412E6102C369AECA2CED933A1DE74712CCDA5EDAF4EE57',
         'EFC2B3DB87FF4F00C79DFA8F732A23C0E18587A73A839B7710234583CDD03DB9',
         'F1B6FE8FC2AB800E6D76EE975A002D3E67A60B51A62085A07289505B8D03F149',
         'E827331B1FE7A2689FBC23D14CD21317C699596CBCA222182A489322ECE1FA74']

In [12]:
def check_root(transaction, proof, depth, pos):
    pos = 2**depth+pos
    count = 0

    while pos>1:
        if pos%2 == 0:
            # Left
            input = transaction + proof[count]
        else:
            # Right
            input = proof[count] + transaction

        transaction = doubleSha256(input)
        print(pos, count, transaction)

        pos = pos // 2
        count += 1

    return invert_byteorder(transaction)

In [13]:
root = check_root(transactions[6], proof, 4, 6)

22 0 066ad6d9939ec0c90b7f3b775122785bd8aca2a2c5857205af7e0615e9af9796
11 1 8be15fc2ab11ef3e079568d43b2b09ed5a5690fb13ecb1032f7aab99238a1847
5 2 8d9d737b484e96eed701c4b3728aea80aa7f2a7f57125790ed9998f9050a1bef
2 3 b152eca4364850f3424c7ac2b337d606c5ca0a3f96f1554f8db33d2f6f130bbe


In [14]:
root == block["mrkl_root"]

True

Here we had only to perform 4 calculations instead of the original 11.

# Hash a block header

Blocks are represented by the doube sha256 hash of their header contents. Verifying the hash is relatively straight forward. We must simply pack all the relevant information in the correct sequence and then perform the double SHA256 calculation.

In [15]:
def block_hash(block):
    # Header version "01000000"
    header = "01000000" + \
             invert_byteorder(block["prev_block"]) +\
             invert_byteorder(block["mrkl_root"]) + \
             invert_byteorder(hex(block["time"])[2:]) + \
             invert_byteorder(hex(block["bits"])[2:])

    if block["nonce"] < 0:
        header += invert_byteorder(hex(2**32+block["nonce"])[2:]) # Convert to unsigned
    else:
        header += invert_byteorder(hex(block["nonce"])[2:])

    return invert_byteorder(doubleSha256(header))

For this particular block, we have:

In [36]:
blockhash = block_hash(block)
print(blockhash)

0000000000000a3290f20e75860d505ce0e948a1d1d846bec7e39015d242884b


And this is indeed the correct value

In [37]:
blockhash == block["hash"]

True

We should also note that any change to the block header contents will invalidate the hash by a random amount. For example, if we modify the **nonce** even by a small amount, we obtain something completely different

In [40]:
block["nonce"] -= 1

In [42]:
new_blockhash = block_hash(block)
print(new_blockhash)

e38fdcc6c7fc14dde8f72300c3d89754b255ec6acbc6c4589fd9052a5eb472f7


# Proof of work

As we just saw, miners "pay" in computational time by searching for the correct value of the nonce that results in a sufficiently small hash for the entire block. The block difficulty is updated regularly and can be found in the "bits" field of the block.

In [43]:
difficulty = hex(block["bits"])

This is a bit packed field where the first 4 bytes define the exponent and the remainig digits the coefficient value

In [32]:
hash_target = difficulty[4:] # Coefficient
hash_target += "00"*(int(difficulty[:4], 0)-3) # Pad with the zeros on the right
hash_target = "0"*(64-len(hash_target)) + hash_target # Add missing zeros on the left for expected size 

In Hex (and dec), this the number we are trying to beat. The miner will keep modifying the "nonce" value until the hash of the block is **smaller** than this value.

In [33]:
print(hash_target, int("0x"+hash_target, 0))

0000000000000b6d4b0000000000000000000000000000000000000000000000 18362361570655080300849714079315004638119732162003921272832000


The hash of the current block is:

In [34]:
print(block["hash"], int("0x"+block["hash"], 0))

0000000000000a3290f20e75860d505ce0e948a1d1d846bec7e39015d242884b 16386789583490230259545288182434459053535275237905118869489739


and we can easily verify that indeed the block hash is below this value

In [26]:
int("0x"+hash_target, 0) > int("0x"+block["hash"], 0)

True

So we can be confident that this is indeed a valid block